In [1]:
!pip install --upgrade pytube youtube-comment-downloader pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.


In [2]:
!pip install youtube-comment-downloader
!pip install youtube-search-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.5/99.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 8.4 MB/s eta 0:00:00


In [6]:
# Import library yang dibutuhkan
import pandas as pd
from youtube_comment_downloader import YoutubeCommentDownloader
from pytube import Search  # Menggunakan Pytube untuk pencarian
import itertools

# -----------------------------------------------------
# --- PENGATURAN SCRIPT (SILAKAN UBAH BAGIAN INI) ---
# -----------------------------------------------------

# 1. Masukkan KATA KUNCI yang ingin Anda cari di YouTube
KATA_KUNCI_PENCARIAN = "ulasan Toyota Vios 2023 Indonesia"

# 2. Tentukan berapa video teratas dari hasil pencarian yang ingin di-scrape
JUMLAH_VIDEO_UNTUK_DISCRAPE = 10 # Ambil 3 video teratas

# 3. Batasi jumlah maksimal komentar yang diambil PER VIDEO
MAKSIMAL_KOMENTAR_PER_VIDEO = 200 # Ambil 150 komentar dari setiap video

# 4. Tentukan nama file output untuk menyimpan SEMUA komentar
NAMA_FILE_OUTPUT = 'hasil_scraping_vios_pytube.csv'

# -----------------------------------------------------
# --- PROSES SCRAPING OTOMATIS (REVISI) ---
# -----------------------------------------------------

print(f"Mencari video di YouTube dengan kata kunci: '{KATA_KUNCI_PENCARIAN}'...")

# TAHAP 1: MENCARI VIDEO MENGGUNAKAN PYTUBE
try:
    s = Search(KATA_KUNCI_PENCARIAN)
    list_url_video = []
    # Ambil URL video sebanyak JUMLAH_VIDEO_UNTUK_DISCRAPE
    for i, video in enumerate(s.results):
        if i >= JUMLAH_VIDEO_UNTUK_DISCRAPE:
            break
        list_url_video.append(video.watch_url)
except Exception as e:
    print(f"Gagal melakukan pencarian video. Error: {e}")
    list_url_video = []


if not list_url_video:
    print("Tidak ada video yang ditemukan dengan kata kunci tersebut.")
else:
    print(f"Berhasil menemukan {len(list_url_video)} video. Memulai proses scraping komentar...")

    semua_komentar_total = []
    downloader = YoutubeCommentDownloader()

    # TAHAP 2: SCRAPING KOMENTAR DARI SETIAP VIDEO (Tetap sama)
    for i, url in enumerate(list_url_video):
        print(f"\n({i+1}/{len(list_url_video)}) Scraping dari URL: {url}")
        try:
            komentar_generator = downloader.get_comments_from_url(url, sort_by=1)
            for comment in itertools.islice(komentar_generator, MAKSIMAL_KOMENTAR_PER_VIDEO):
                data_komentar = {
                    'username': comment.get('author'),
                    'waktu': comment.get('time'),
                    'teks_komentar': comment.get('text'),
                    'jumlah_likes': comment.get('votes', 0),
                    'sumber_video': url
                }
                semua_komentar_total.append(data_komentar)
            print(f"  -> Selesai. Berhasil mengambil komentar dari video ini.")
        except Exception as e:
            print(f"  -> Gagal mengambil komentar dari video ini. Error: {e}")
            continue

    # TAHAP 3: MENYIMPAN HASIL GABUNGAN (Tetap sama)
    if semua_komentar_total:
        df_gabungan = pd.DataFrame(semua_komentar_total)
        df_gabungan.to_csv(NAMA_FILE_OUTPUT, index=False)
        print(f"\n✅ PROSES SELESEI! Total {len(df_gabungan)} komentar dari {len(list_url_video)} video telah disimpan ke '{NAMA_FILE_OUTPUT}'")
    else:
        print("\nTidak ada komentar yang berhasil di-scrape.")

Mencari video di YouTube dengan kata kunci: 'ulasan Toyota Vios 2023 Indonesia'...


Berhasil menemukan 10 video. Memulai proses scraping komentar...

(1/10) Scraping dari URL: https://youtube.com/watch?v=HR4ELY1Xm3w
  -> Selesai. Berhasil mengambil komentar dari video ini.

(2/10) Scraping dari URL: https://youtube.com/watch?v=FDU3zV2_n04
  -> Selesai. Berhasil mengambil komentar dari video ini.

(3/10) Scraping dari URL: https://youtube.com/watch?v=dCynfd001QM
  -> Selesai. Berhasil mengambil komentar dari video ini.

(4/10) Scraping dari URL: https://youtube.com/watch?v=YdTPa-zESgM
  -> Selesai. Berhasil mengambil komentar dari video ini.

(5/10) Scraping dari URL: https://youtube.com/watch?v=lWlan_59zfI
  -> Selesai. Berhasil mengambil komentar dari video ini.

(6/10) Scraping dari URL: https://youtube.com/watch?v=qWbTS6lwOnQ
  -> Selesai. Berhasil mengambil komentar dari video ini.

(7/10) Scraping dari URL: https://youtube.com/watch?v=FhNVPXJW_g8
  -> Selesai. Berhasil mengambil komentar dari video ini.

(8/10) Scraping dari URL: https://youtube.com/watch?v=wLEeN

In [7]:
import pandas as pd

# 1. Baca file CSV yang sudah Anda buat sebelumnya
nama_file = 'hasil_scraping_vios_pytube.csv'
df = pd.read_csv(nama_file)

# 2. Terapkan case folding (mengubah ke huruf kecil) pada kolom 'teks_komentar'
#    Kita akan membuat kolom baru bernama 'komentar_bersih' untuk menyimpan hasilnya
#    agar data asli tetap ada jika diperlukan.
df['komentar_bersih'] = df['teks_komentar'].str.lower()

# 3. Tampilkan perbandingan sebelum dan sesudah case folding untuk 5 baris pertama
print("Perbandingan Sebelum dan Sesudah Case Folding:")
print(df[['teks_komentar', 'komentar_bersih']].head())

# 4. (Opsional) Simpan hasilnya ke file CSV baru jika Anda mau
#    File ini akan berisi semua kolom lama ditambah kolom 'komentar_bersih'
df.to_csv('hasil_vios_1_casefolding.csv', index=False)

print("\n✅ Proses case folding selesai.")

Perbandingan Sebelum dan Sesudah Case Folding:
                                       teks_komentar  \
0                         sayang gada manual nya.. 😢   
1  Keren sih di harga Rp300jt-an sudah dapat mobi...   
2                      cakep ya, tp sedikit di jalan   
3                          Empuk,enak buat bawa tamu   
4                      Cek Harga & pajak tahunan nya   

                                     komentar_bersih  
0                         sayang gada manual nya.. 😢  
1  keren sih di harga rp300jt-an sudah dapat mobi...  
2                      cakep ya, tp sedikit di jalan  
3                          empuk,enak buat bawa tamu  
4                      cek harga & pajak tahunan nya  

✅ Proses case folding selesai.
